## Lecture 02  
## Python Performance Tips  
### Feb. 01, 2023

## Useful resources
- http://alberto.bietti.me/python-performance-tips/

- https://wiki.python.org/moin/PythonSpeed/PerformanceTips

- Python is a dynamic interpreted language (not a compiled language)

- It is not compiled to the native object code and executed on a computer system

- **Types** of variables, function arguments, etc. are not known until the program runs

- Dynamic interpreted languages have great flexibility, but suffer significant performance limitations

- Difficult to optimize, dependence on the interpreter


---

- Python is easy to learn, write, read, debug

- A large library of built-in functions and libraries: https://docs.python.org/3/library/functions.html

---


### How to optimize?

- Get the program to give correct results

- Then rerun to see if the correct program is slow

- Profile to find which parts of the program consume most of the time 

- Repeat

### Today's topics: 

- Built-in functions

- Function Call Overhead

- Function Decorator

- Loops, and built-in operators

- Membership operator **in** 

- lists lookup time O(n), hash (dicts, sets) are O(1)


## Timing Python Code

In [2]:
# manually to time:

import time 

start_time = time.time() ## records the current time. 

#factorial 500! = 1 * 2 * 3 * ... * 500
fact = 1
for i in range(1, 500): 
    fact *= i

end_time = time.time() ## records the current time  (but effectlivly end time)

# print(fact)

print("run_time: %f" % (end_time - start_time)) ## prints run time. 

run_time: 0.000166


In [3]:
import time
print(time.time()) ## this is the unix time stamp, there is a conversion method

1675914344.2598224


In [4]:
# Timing Python Code
# timeit module
# To see how long it takes a program to run once;
# on average over a bunch of runs, e.g. over k=10000 runs;

import timeit

def my_function():
    fact = 1
    for i in range(1, 500): 
        fact *= i


k = 10000
print("run_time:", timeit.timeit(my_function, number=k) / k) ## the timeit fucntion will implemet timining better. 


# median can avoid outliers. 
# if some runs take a really long time, can use median. 

run_time: 3.4784596899953616e-05


In [5]:
#if using IPython

# %timeit my_function()

%timeit -n 1000 -r 7 my_function() ## percentage symbol can be used as magic functions https://ipython.readthedocs.io/en/stable/interactive/magics.html

34.9 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
#if using IPython

%timeit -r 10 my_function()

%timeit -r 10 -n 1000 my_function()



34.3 µs ± 724 ns per loop (mean ± std. dev. of 10 runs, 10,000 loops each)
33.7 µs ± 344 ns per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [7]:
def f(x):
    return x**2

def g(x):
    return x**4

def h(x):
    return x**8


%timeit -n 10000 f(5)

%timeit -n 10000 g(5)

%timeit -n 10000 h(5)

## we can use this a lot. 

219 ns ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
194 ns ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
201 ns ± 3.04 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Built-in Functions


- One of the easiest ways to improve Python performance is not to execute any Python code at all! 

- Python provides a large number of built-in functions that perform a wide variety of operations. 

- These built-in functions are written in C, and so are generally very fast. 

- See the Python documentation for a list of the available functions: https://docs.python.org/3/library/functions.html


In [8]:
import random

def my_min(values):
    min_value = values[0]
    for v in values:
        if v < min_value:
            min_value = v
    return min_value


random_numbers = [random.random() for _ in range(0,100_000)] ## finding min amont 100,000 values 

print(my_min(random_numbers), min(random_numbers))

#time "my_min()" ## this si an O(n) algoryhm
%timeit -n 100 my_min(random_numbers)

#IPython already provides the function "min()"
%timeit -n 100 min(random_numbers)


## use built in functions in if you can,the code are generally well written.  

## when writing functions, it should first of all be right, then compare the time to a build in. 

1.1504611368673423e-05 1.1504611368673423e-05
1.44 ms ± 49.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
756 µs ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
import random
print(random.random())

## generaete a random number

0.519778143497804


# Function Call Overhead  


**How functions affect a program’s performance?**  

- Function call overhead in Python is relatively high, especially compared with the execution speed of builtin functions. 

- The overhead is due, e.g., to loading function objects, loading and checking function arguments, dynamic type checking of function arguments that must be performed before and after the function call.

- One idea is to minimize the number of function calls by handling aggregates


- calling functions in pyhton is not free. so dont call a funciton a lot of times. 
- when ever you call a funciton need to load the function into memory, check argumetns etc. 
- so if it can be avoided dont call a function many times. 
- functions can be input arugmetns to other functions in pyhton becuase they are objects .

In [10]:
total_sum = 0

def inner(i): ## 
    global total_sum ## gloabl makes sure we have acess to that specfic value
    total_sum += i ## increments by a vlaue of i. 

def outer_1(): 
    for i in range(10_000 + 1): 
        inner(i) ## for i in range do a sum up to that value
outer_1()         ## so there are 10_001 function calls. 
total_sum

50005000

In [11]:
%%timeit -n 100

total_sum = 0
def inner(i):
    global total_sum
    total_sum += i
    
# The sum of the first n non-negative integers
# S_{n} = 1 + ... + n 

def outer_1():
    for i in range(10_000 + 1): 
        inner(i)
        
outer_1()
## thi si slow. 

731 µs ± 41.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


- The "inner" function was called 10000 times. 

- Instead, move the loop inside the "aggregate" function and call it only once!

In [12]:
# %%timeit -n 100

def aggregate(l): ## l is a list instead of a number, data is agregated in this funciton 
    ## generator yield key word is good. 
    x = 0 
    for i in l:
        x = x + i
    return x

def outer_2():
    return aggregate(range(10_000 + 1)) ## so runs teh same thing on a list
    
%timeit -n 100 outer_2()
print(outer_2()) ## does this sigfantly faster ebcuase it has 2 functions calls. 

279 µs ± 9.24 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
50005000


## Membership Testing

- Python provides the **in** operator (a membership operator) to check if an element exists in a collection. 

- The **in** operator is very fast at checking if an element exists in a **dict** or a **set**, because both dict and set are implemented using a **hash table**. 
- but slow to check membership within lists .

In [13]:
letters = 'abcdefghijklmnopqrstuvwxyz' ## strings are kinda like lists so membership cehcking here is O(n), if you know the length of strings you can just add to a set or dict
letters_list = [x + y + z for x in letters for y in letters for z in letters]  ## composes all of the three letter combinations of the letters list. 
## list comphresiion is faster than sepeate for loops. 
print("first 10 members:", letters_list[:10])
print("last  10 members:", letters_list[-10:])

first 10 members: ['aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'aai', 'aaj']
last  10 members: ['zzq', 'zzr', 'zzs', 'zzt', 'zzu', 'zzv', 'zzw', 'zzx', 'zzy', 'zzz']


In [14]:
len(letters_list)

17576

In [15]:
print("len_letters_list = %d" % len(letters_list))
print("len_letters ** 3 = %d = %d ** 3" % (len(letters) ** 3, len(letters)))

len_letters_list = 17576
len_letters ** 3 = 17576 = 26 ** 3


In [16]:
"aaa" in letters_list

True

In [17]:
"zzz" in letters_list

True

In [18]:
#Membership of a List
%timeit ("aaa" in letters_list) ## frist ellement in lit
%timeit ("mmm" in letters_list)
%timeit ("zzz" in letters_list) ## last elelemnt in list takes 10 times as loong 

22.1 ns ± 0.195 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
79.1 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
159 µs ± 405 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Checking for membership in a list or tuple is not as efficient!

In [19]:
#Membership of a Dictionary

# identity mapping: 
letters_dict = dict([(x, x+'a') for x in letters_list]) ## this is not a space efficent sollution but is faster still .

for k, v in letters_dict.items(): ## goes throuugh keys and values. 
    print(k, ":", v) 

aaa : aaaa
aab : aaba
aac : aaca
aad : aada
aae : aaea
aaf : aafa
aag : aaga
aah : aaha
aai : aaia
aaj : aaja
aak : aaka
aal : aala
aam : aama
aan : aana
aao : aaoa
aap : aapa
aaq : aaqa
aar : aara
aas : aasa
aat : aata
aau : aaua
aav : aava
aaw : aawa
aax : aaxa
aay : aaya
aaz : aaza
aba : abaa
abb : abba
abc : abca
abd : abda
abe : abea
abf : abfa
abg : abga
abh : abha
abi : abia
abj : abja
abk : abka
abl : abla
abm : abma
abn : abna
abo : aboa
abp : abpa
abq : abqa
abr : abra
abs : absa
abt : abta
abu : abua
abv : abva
abw : abwa
abx : abxa
aby : abya
abz : abza
aca : acaa
acb : acba
acc : acca
acd : acda
ace : acea
acf : acfa
acg : acga
ach : acha
aci : acia
acj : acja
ack : acka
acl : acla
acm : acma
acn : acna
aco : acoa
acp : acpa
acq : acqa
acr : acra
acs : acsa
act : acta
acu : acua
acv : acva
acw : acwa
acx : acxa
acy : acya
acz : acza
ada : adaa
adb : adba
adc : adca
add : adda
ade : adea
adf : adfa
adg : adga
adh : adha
adi : adia
adj : adja
adk : adka
adl : adla
adm : adma

In [20]:
# "aaa" in letters_dict
# "zzz" in letters_dict

In [21]:
%timeit ("aaa" in letters_dict)
%timeit ("mmm" in letters_dict) ## aproxmentally constnat look up times. within dicts, becuase they are hash maps. 
%timeit ("zzz" in letters_dict)

21.8 ns ± 0.174 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
22.1 ns ± 0.473 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
23 ns ± 0.321 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### You could also convert a list into a set and check for a membership

---

In [22]:
letters_set = set(letters_list) ## here we are using set's

%timeit ("aaa" in letters_set) ## these also have constant look up time. 
%timeit ("mmm" in letters_set) ## sets and dictionaries have no order, there is a special data strcture called orderd_set and orderd_list
%timeit ("zzz" in letters_set)

20.8 ns ± 0.417 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
20.6 ns ± 0.0726 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
25.9 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [23]:
dd = {'a': 1, 'b': 2}
ss = {'a', 'b'} 
print('a' in dd, 'b' in ss)


True True


## String Concatenation

In [24]:
def make_string(string_list): ## list with cahricters, 
    my_string = ''
    for character in string_list: ## look through char list
        my_string += character  ## append charicters to stsring 
    return my_string

str_list = [character for character in 'abcdefghijklmnopqrstuvwxyz'] ## will make liist out of this string 
print(str_list)
print(make_string(str_list)) ## will make a string out of this list

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
abcdefghijklmnopqrstuvwxyz


In [25]:
%timeit make_string(str_list) ## pretty slow 

778 ns ± 16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [26]:
%timeit "".join(str_list) ## a lot faster. 
## use join metho, which is also a biild in

140 ns ± 3.51 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [27]:
','.join(['a', 'b', 'c'])

'a,b,c'

In [28]:
'a,b,c'.split(',') ## oppesite operator of join. 

## when ever possible just use built in fnctions. 

['a', 'b', 'c']

# Decorator Caching

## Function Decorator



---

- The symbol **@** is Python decorator syntax. 


- Python decorators are callable Python object that is used to modify a function, method or class definition.   


- Python decorators are normally used for tracking, locking, or logging  


- The wise use of decorators can improve the performance of codes.  


- Decorate a Python function so that it remembers the results needed later  

---

- the main pupose is that @ is a decerator
- we need  a decerator to allow us to extend a function, with out chaning the original behavior
- is not super usefull, but should know what it does.
- decerators can improve the preformance of code. 
- decerating functions can also help save ressults of a function

In [29]:
def decorating_a_function(func): ##decerator fucntion takes as input a function 
    def function_wrapper(x): ## function wrapper, 
        print("Now \"" + func.__name__ + "\" becomes decorated.")
        print("The attribute:")
        func(x)
        print("is surrounded by all these text!")
    return function_wrapper #3 allows us to expand the function and returns the new function  

def foo(x):
    print(x) ## this will just print the input

In [30]:
# run the function
foo("test")

# separator
print("-" * 50)

# get the function name
print("function_name:", foo.__name__)

test
--------------------------------------------------
function_name: foo


In [31]:
foo = decorating_a_function(foo) ## renaming foo as that new function 

# run the function
foo("hello")

# separator
print("-" * 50)

# get the function name
print(foo.__name__) ## the name of foo has also changed to be name function wrapper. 
#

Now "foo" becomes decorated.
The attribute:
hello
is surrounded by all these text!
--------------------------------------------------
function_wrapper


In [32]:
def decorating_a_function(func):
    def function_wrapper(x):
        print("Now the function called \"" + func.__name__ + "\" becomes decorated.")
        print("The attribute of the function:")
        func(x)
        print("is surrounded by all this text!")
    return function_wrapper

@decorating_a_function ## this does the same thing as  def decorating_a_function(func):
def foo(x):
    print(x)

In [33]:
# Test 1:
foo("test")

Now the function called "foo" becomes decorated.
The attribute of the function:
test
is surrounded by all this text!


In [34]:
# Test 2:
print(foo.__name__)

function_wrapper


## Imported functions can also be decorated

In [35]:
from math import sin, cos, pi

def our_decorator(func):
    def function_wrapper(x):
        val = func(x)
        print("The result of %s(%0.4f) is: %0.4f" % (func.__name__, x, val)) ## retunrs the fucntion in a slightly nicer way 
        return val
    return function_wrapper

# in this case is not possible to use @ becuase they are built in functions. 
sin = our_decorator(sin)
cos = our_decorator(cos)

for f in [sin, cos]: f(pi/2)

for f in [sin, cos]: f(pi)
    
## module is jsut where the function is stored. 

The result of sin(1.5708) is: 1.0000
The result of cos(1.5708) is: 0.0000
The result of sin(3.1416) is: 0.0000
The result of cos(3.1416) is: -1.0000


In [36]:
from math import exp
print(exp.__module__)
exd=our_decorator(exp)
print(exd.__module__)

math
__main__


## Using wraps from module functools

- a module with higher-order functions and operations on callable objects

In [37]:
from functools import wraps

def greeting(func):
    @wraps(func) ## it more or less just changes the objects for the funciton liek the anme 
    def function_wrapper(x):
        """function_wrapper of greeting""" ## there are doc strings. 
        print("Hello, this function " + func.__name__ + " at value " + str(x) + " returns:", func(x))
    
    return function_wrapper

@greeting ## decerating using this greeting function 
def simple_f(x):
    """add 500"""
    return (x + 500)

In [38]:
#call simple_f
simple_f(10)



Hello, this function simple_f at value 10 returns: 510


In [39]:
print("function name: " + simple_f.__name__)
print("docstring: " + simple_f.__doc__)
print("module name: " + simple_f.__module__)

function name: simple_f
docstring: add 500
module name: __main__


In [40]:
from functools import wraps

def greeting(func):
    def function_wrapper(x):
        """function_wrapper of greeting"""
        print("Hello, this function " + func.__name__ + " at value " + str(x) + " returns:", func(x))
    return function_wrapper

@greeting
def simple_f(x):
    """add 500"""
    return (x + 500)

In [41]:
print("function name: " + simple_f.__name__)
print("docstring: " + simple_f.__doc__)
print("module name: " + simple_f.__module__)

function name: function_wrapper
docstring: function_wrapper of greeting
module name: __main__


# Using Decorators for Caching

In [42]:
import time

# Consider Fibonacci numbers: 
# defined as f_n = f_{n - 1} + f_{n - 2} for n >=2 
# where f_0 = 0 and f_1 = 1
# https://en.wikipedia.org/wiki/Fibonacci_number
    

# a simple recursion: 
def fib(i):
    if i <= 1: return i
    return fib(i - 1) + fib(i - 
    2) ## this is slow, you have to keep recomputing the smae number many times. can do this with memoization 
#0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...

t = time.process_time()
fib_result = fib(30)
elapsed_time = time.process_time() - t
print("fibonacci time: %0.10f; fib_result: %d" % (elapsed_time, fib_result))

fibonacci time: 0.1357768000; fib_result: 832040


# Memoization

In [43]:
# The idea is a memoization: 
# introduce a map (dictionary) "memo"
# in which to save intermediate steps
# of calculations

def fib_memo(i, memo=dict()):
    if i <= 1: return i
    if i in memo: return memo[i] ## done using a dictionary not a list. 
    memo[i] = fib_memo(i - 1, memo) + fib_memo(i - 2, memo)
    return memo[i]

t = time.process_time() 
fib_m = fib_memo(100)
elapsed_time_memo = time.process_time() - t
print("fibonacci time: %0.10f; fib_result: %d" % (elapsed_time_memo, fib_m)) #using memoization this is a lot faster. 

fibonacci time: 0.0000877000; fib_result: 354224848179261915075


In [44]:
# We can create a decorator that saves:
# each intermediate value in memory 
# rather than calculating it every time.

from functools import wraps

def cache(f):
    memo = {} ## has a memory dictionary
    @wraps(f) ## wraps keep orignal function behavior
    def function_wrapper(*arg): ## takes a varible number of arguments 
        if arg not in memo:  ## if the argument nut in memoized dict
            memo[arg] = f(*arg) ## then use the orginal function to caclaute it
        return memo[arg] ## return that 
    
    return function_wrapper## retuns the output

@cache ## using the cache decerator 
def fib_cache(i):
    if i < 2: return i
    return fib_cache(i - 1) + fib_cache(i - 2)
## ends up ebing faster

t = time.process_time() 
fib_c = fib_cache(30)
elapsed_time_c = time.process_time() - t
print("fibonacci time: %0.10f; result: %d" % (elapsed_time_c, fib_c))

fibonacci time: 0.0000489000; result: 832040


In [45]:
def add(*numbers):
    return sum(numbers)
 ## here is a thinkg about varible number of argumetns. 
print(add(2, 3))
print(add(2, 3, 5))
print(add(2, 3, 5, 7))
print(add(2, 3, 5, 7, 9))
## there is a trade off between flexability speed and simplicty. 

5
10
17
26


# Optimizing Loops

In [46]:
import random
 ## for loops expensive in pyhton 
lowerlist = ['abcdefghijklmnopqrstuvwxyz' [:random.randint(0, 26)] for x in range(1000)]
upperlist = []

# get firs 20 elements
lowerlist[ : 20]

['abcdefghijklmnopqrstuvwxy',
 'abcdefghijklmnopqrs',
 '',
 '',
 'ab',
 'abcdefghijkl',
 'abcdefghijklmn',
 'abcdefghijklmnopqrst',
 'abcdefghi',
 'ab',
 'abcdefghijklm',
 'abcdefg',
 'abcdefghijklm',
 'abcdefghijklmnopqrst',
 'abcdefghijklmn',
 'abcdefgh',
 'abcdefghijklmnopqr',
 'abcdefghijklmnop',
 'abcdefghijklmnopq',
 'abcdefghijklm']

### Task: From the lowerlist build the upperlist

In [47]:
print('hello'.upper()) ## this is build in upper function 
print(str.upper('hello'))

HELLO
HELLO


In [48]:
len(lowerlist) ## build in function len 

1000

In [49]:
upperlist = []

def to_upper_1():
    for word in lowerlist:
        upperlist.append(str.upper(word))
    return upperlist ## this will do an upper version of lowe list. 

%timeit -n 1000 to_upper_1()

106 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


- The loop calls two methods: "upperlist.append" and "str.upper" every time. 

- Python must support dynamic attributes as well as multiple namespaces. 


In [50]:
upperlist = []
f_upper = str.upper ## store this in a function, and use it. kind of similar to why calling inv() faster than np.liniag.inv
f_append = upperlist.append ## so import the specfic functions more, and it will lead to large preformance increases. 

def to_upper_2():
    for word in lowerlist:
        f_append(f_upper(word))

%timeit -n 1000 to_upper_2()

62.6 µs ± 4.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [51]:

def to_upper_3():
    upperlist = [] 
    f_upper = str.upper
    f_append = upperlist.append

    for word in lowerlist:
        f_append(f_upper(word))

%timeit -n 1000 to_upper_3()

49.5 µs ± 989 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Avoid the loop

In [52]:
# A "map" is often called "apply-to-all" when considered in functional form;
# e.g. a map applied on all elements of a list

# def f(x):
#     return x + 100

simple_mapping = map(lambda x : x + 100, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) ## map like in math, 
## map lambda will map x to x+100, this will make  astring map, 

list(simple_mapping)

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]

In [53]:
# Apply the method "upper" on strings

print(lowerlist[0], str.upper(lowerlist[0]))

abcdefghijklmnopqrstuvwxy ABCDEFGHIJKLMNOPQRSTUVWXY


In [54]:

# avoiding the loop by using "map"
upper = str.upper
%timeit -n 1000 upperlist = list(map(upper, lowerlist)) ### this outputs the same thing but avoids the for loop, using the map . 

32.6 µs ± 692 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [55]:

# avoiding the loop by using "list comprehension"
f_upper = str.upper 
%timeit -n 1000 upperlist = [f_upper(word) for word in lowerlist] ## could also use list comphrepsion, which will help aboid loops. 

39.7 µs ± 3.59 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [56]:
# add two random vectors; given as lists

import random 

random_numbers1 = [random.random() for _ in range(0,100000)]
random_numbers2 = [random.random() for _ in range(0,100000)]

%timeit res1 = list(map(lambda x, y: x + y, random_numbers1, random_numbers2))

# However map is calling our function as "adding two vectors" not as "cross product" 

4.83 ms ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
len(list(map(lambda x, y: x + y, random_numbers1, random_numbers2)))

100000

## Intrinsic Opertors

- Another performance improvement is to use intrinsic operators (+, -, *, etc.) instead of a user defined function.  


- The operator module exports a set of efficient functions corresponding to the intrinsic operators of Python.   


- operator.add(x, y) is equivalent to the expression x + y.   


- `import operator`

https://docs.python.org/3.4/library/operator.html

In [58]:
import operator
## these opperators are faster than 
%timeit len(list(map(lambda x, y: x + y, random_numbers1, random_numbers2)))
%timeit res2 = list(map(operator.add, random_numbers1, random_numbers2)) ## this operator.add is a lot faster than otherwirs.e

5.16 ms ± 258 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.39 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### using maps

- In Python 3.5, **the map function returns an iterator that does not evaluate the arguments until it needs to**. 

-  By converting the iterator to a list, we are forcing map to compute every value.


In [59]:
#Instead here we use the operator "add" directly without "map":

%timeit res3 = operator.add(random_numbers1, random_numbers2)

499 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
# Using numpy: exploit contiguous memory, special instructions

## this is faster becuse this has contiogous memory as well as numpy optimizations that make things much faster as well as primary data type. 
import numpy as np

rand1_np = np.array(random_numbers1)
rand2_np = np.array(random_numbers2)

%timeit res4 = rand1_np + rand2_np
## for array operations use numpy is take away

53.6 µs ± 671 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [61]:
# Using Python arrays (if you don't have Numpy...) 


import array

rand1_arr = array.array('d', random_numbers1)
rand2_arr = array.array('d', random_numbers2)

%timeit res5 = operator.add(rand1_arr, rand2_arr) ## a direct array is pretty similar to usig numpy. 

46 µs ± 887 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
